# ライトニングネットワーク中心性分析

このノートブックでは、ビットコインのライトニングネットワークにおける3つの中心性指標を分析します：

1. **媒介中心性 (Betweenness Centrality)**: ノードが他のノード間の最短経路にどれだけ頻繁に現れるか
2. **近接中心性 (Closeness Centrality)**: ノードから他のすべてのノードへの平均距離の逆数
3. **近似中心性 (Eigenvector Centrality)**: 重要なノードに接続されているノードが重要とされる

## 目的

どのノードとチャネルを開設したらルーティングがされやすくなるかを分析します。


## 0. 依存パッケージのインストール

**重要**: 初回実行時、またはパッケージが更新された場合は、このセルを実行してください。


In [ ]:
# requirements.txtからすべてのパッケージをインストール
# 注意: インストール後はカーネルを再起動してください

import os
import sys
import subprocess

# プロジェクトルートを取得
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
requirements_path = os.path.join(project_root, 'requirements.txt')

if os.path.exists(requirements_path):
    print(f"requirements.txtを読み込みます: {requirements_path}")
    print("パッケージをインストールしています...")
    
    # subprocessを使用してpip installを実行
    # %pipマジックコマンドでは変数展開ができないため、subprocessを使用
    try:
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'install', '-r', requirements_path],
            capture_output=True,
            text=True,
            check=False
        )
        
        if result.returncode == 0:
            print(result.stdout)
            print("\n✓ パッケージのインストールが完了しました")
            print("⚠️  重要: カーネルを再起動してください（Kernel → Restart Kernel）")
        else:
            print("エラーが発生しました:")
            print(result.stderr)
            print("\n代替方法: 以下のコマンドを手動で実行してください")
            print(f"  %pip install -r {requirements_path}")
    except Exception as e:
        print(f"インストール中にエラーが発生しました: {e}")
        print("\n代替方法: 以下のコマンドを手動で実行してください")
        print(f"  %pip install -r {requirements_path}")
else:
    print(f"✗ requirements.txtが見つかりません: {requirements_path}")
    print(f"現在のディレクトリ: {os.getcwd()}")
    print(f"プロジェクトルート: {project_root}")


## 1. 環境設定とインポート


In [ ]:
import sys
import os

# プロジェクトルートをパスに追加
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from pathlib import Path

# ロギング設定
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# 日本語フォント設定（必要に応じて）
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")

print("環境設定が完了しました")


## 2. データベース接続とデータ取得


In [ ]:
from src.database.connection import DatabaseConnection
from src.graph.builder import GraphBuilder
import networkx as nx

# データベース接続
db = DatabaseConnection()

# グラフ構造情報を取得
graph_info = db.get_graph_structure()
print(f"グラフ構造情報:")
print(f"  ノード数: {graph_info['node_count']}")
print(f"  エッジ数: {graph_info['edge_count']}")

# グラフを構築（基本グラフ、重みなし）
# 注意: トランザクションサイズを考慮した分析は、次のセクションで実行します
graph_builder = GraphBuilder(db)
graph = graph_builder.build_graph(directed=False)

print(f"\n基本グラフが構築されました:")
print(f"  ノード数: {graph.number_of_nodes()}")
print(f"  エッジ数: {graph.number_of_edges()}")
print(f"  連結成分数: {nx.number_connected_components(graph)}")

# 論文 "A Centrality Analysis of the Lightning Network" (2201.07746v1) を参照
# 異なるトランザクションサイズでルーティング手数料が変わるため、
# 中心性スコアも変化する可能性がある
print("\n注意: トランザクションサイズを考慮した分析は、次のセクションで実行します")


## 2.1 データの妥当性確認

取得したデータが正しいか、mempool.spaceやAmbossなどのLightning Network統計と比較して確認します。


In [ ]:
# データの妥当性を確認
# mempool.spaceやAmbossなどのLightning Network統計と比較

# 1. ユニークなチャネルIDの数を確認
unique_channels_query = """
SELECT COUNT(DISTINCT chan_id) as unique_channels
FROM channel_update
WHERE rp_disabled = false
AND chan_id NOT IN (SELECT chan_id FROM closed_channel)
"""
result = db.execute_query_single(unique_channels_query)
print(f"✓ ユニークなチャネル数: {result['unique_channels']:,}")

# 2. 最新の更新のみを取得した場合のチャネル数
latest_channels_query = """
SELECT COUNT(*) as latest_channels
FROM (
    SELECT DISTINCT ON (chan_id) chan_id
    FROM channel_update
    WHERE rp_disabled = false
    AND chan_id NOT IN (SELECT chan_id FROM closed_channel)
    ORDER BY chan_id, timestamp DESC
) latest
"""
result2 = db.execute_query_single(latest_channels_query)
print(f"✓ 最新の更新のみ（重複除外）: {result2['latest_channels']:,}")

# 3. データの最新性を確認（最新の更新日時）
latest_timestamp_query = """
SELECT MAX(timestamp) as latest_timestamp,
       MIN(timestamp) as oldest_timestamp,
       COUNT(*) as total_updates
FROM channel_update
WHERE rp_disabled = false
AND chan_id NOT IN (SELECT chan_id FROM closed_channel)
"""
result3 = db.execute_query_single(latest_timestamp_query)

# タイムスタンプを日時に変換
from datetime import datetime
latest_dt = datetime.fromtimestamp(result3['latest_timestamp']) if result3['latest_timestamp'] else None
oldest_dt = datetime.fromtimestamp(result3['oldest_timestamp']) if result3['oldest_timestamp'] else None

print(f"\nデータの範囲:")
if latest_dt:
    print(f"  最新の更新: {latest_dt.strftime('%Y-%m-%d %H:%M:%S')}")
if oldest_dt:
    print(f"  最も古い更新: {oldest_dt.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"  総更新数: {result3['total_updates']:,}")

# 4. 最近30日以内の更新のみを取得した場合
recent_channels_query = """
SELECT COUNT(DISTINCT chan_id) as recent_channels
FROM (
    SELECT DISTINCT ON (chan_id) chan_id
    FROM channel_update
    WHERE rp_disabled = false
    AND chan_id NOT IN (SELECT chan_id FROM closed_channel)
    AND timestamp >= EXTRACT(EPOCH FROM NOW() - INTERVAL '30 days')::bigint
    ORDER BY chan_id, timestamp DESC
) recent
"""
result4 = db.execute_query_single(recent_channels_query)
print(f"\n最近30日以内の更新のみ: {result4['recent_channels']:,}")

# 5. ノード数の詳細確認
node_details_query = """
SELECT 
    COUNT(DISTINCT advertising_nodeid) as advertising_nodes,
    COUNT(DISTINCT connecting_nodeid) as connecting_nodes,
    COUNT(DISTINCT CASE 
        WHEN advertising_nodeid IS NOT NULL THEN advertising_nodeid 
        WHEN connecting_nodeid IS NOT NULL THEN connecting_nodeid 
    END) as total_unique_nodes
FROM (
    SELECT DISTINCT ON (chan_id) 
        advertising_nodeid, 
        connecting_nodeid
    FROM channel_update
    WHERE rp_disabled = false
    AND chan_id NOT IN (SELECT chan_id FROM closed_channel)
    ORDER BY chan_id, timestamp DESC
) latest_channels
"""
result5 = db.execute_query_single(node_details_query)
print(f"\nノード数の詳細:")
print(f"  advertising_nodeid: {result5['advertising_nodes']:,}")
print(f"  connecting_nodeid: {result5['connecting_nodes']:,}")
print(f"  ユニークなノード数: {result5['total_unique_nodes']:,}")

# 6. 参考値との比較
print(f"\n=== 参考値との比較 ===")
print(f"Lightning Network全体の統計（2024年10月時点）:")
print(f"  ノード数: 約15,042")
print(f"  チャネル数: 約46,398")
print(f"\n取得データ:")
print(f"  ノード数: {graph_info['node_count']:,} ({graph_info['node_count']/15042*100:.1f}%)")
print(f"  エッジ数: {graph_info['edge_count']:,} ({graph_info['edge_count']/46398*100:.1f}%)")
print(f"\n注意:")
print(f"  - ノード数が少ない場合: 一部のノードが除外されている可能性があります")
print(f"  - エッジ数が多い場合: 同じチャネルの更新が重複している可能性があります")
print(f"  - 最新のデータのみを使用する場合は、日数制限を追加することを検討してください")


## 3. 中心性計算


In [ ]:
from src.centrality.calculator import CentralityCalculator
import numpy as np

# 中心性計算器を初期化
calculator = CentralityCalculator()

# 基本グラフでの中心性計算（重みなし）
print("=" * 60)
print("基本グラフでの中心性計算（重みなし）")
print("=" * 60)
print("中心性計算を開始します...")
centrality_scores = calculator.calculate_all(graph)

print("\n計算が完了しました:")
for name, scores in centrality_scores.items():
    print(f"  {name}: {len(scores)}個のノード")
    if scores:
        values = list(scores.values())
        print(f"    平均: {np.mean(values):.6f}")
        print(f"    最大: {np.max(values):.6f}")
        print(f"    最小: {np.min(values):.6f}")

# 論文 "A Centrality Analysis of the Lightning Network" (2201.07746v1) を参照
# ルーティング手数料は、fee_base_msat + (transaction_size_msat * fee_proportional_millionths / 1_000_000)
# で計算されるため、トランザクションサイズによって中心性スコアが変化する可能性がある


## 3.1 トランザクションサイズを考慮した中心性計算

論文 "A Centrality Analysis of the Lightning Network" (2201.07746v1) を参照すると、
ルーティング手数料は以下の式で計算されます：

**手数料 = fee_base_msat + (transaction_size_msat × fee_proportional_millionths / 1,000,000)**

異なるトランザクションサイズでルーティング手数料が変わるため、
中心性スコア（特に媒介中心性と近接中心性）も変化する可能性があります。

このセクションでは、複数のトランザクションサイズで中心性を計算し、比較します。


In [ ]:
# トランザクションサイズを考慮した中心性計算
# 論文 "A Centrality Analysis of the Lightning Network" (2201.07746v1) を参照

# 異なるトランザクションサイズを定義（millisatoshi）
# 1 satoshi = 1,000 millisatoshi
transaction_sizes = {
    'small': 1_000_000,      # 1,000 satoshi (約 $0.50 @ $50,000/BTC)
    'medium': 10_000_000,    # 10,000 satoshi (約 $5.00)
    'large': 100_000_000,     # 100,000 satoshi (約 $50.00)
}

# 各トランザクションサイズでの中心性を計算
centrality_by_transaction_size = {}

for size_name, size_msat in transaction_sizes.items():
    print(f"\n{'=' * 60}")
    print(f"トランザクションサイズ: {size_name} ({size_msat:,} msat = {size_msat/1_000_000:.0f} sat)")
    print(f"{'=' * 60}")
    
    # このトランザクションサイズでの中心性を計算
    scores = calculator.calculate_all(graph, transaction_size_msat=size_msat)
    centrality_by_transaction_size[size_name] = scores
    
    print(f"\n計算結果:")
    for name, score_dict in scores.items():
        if score_dict:
            values = list(score_dict.values())
            print(f"  {name}:")
            print(f"    平均: {np.mean(values):.6f}")
            print(f"    最大: {np.max(values):.6f}")
            print(f"    最小: {np.min(values):.6f}")

print(f"\n{'=' * 60}")
print("すべてのトランザクションサイズでの計算が完了しました")
print(f"{'=' * 60}")

# 結果を保存（後続の分析で使用）
# デフォルトとして、mediumサイズの結果を使用
centrality_scores = centrality_by_transaction_size.get('medium', centrality_scores)


## 4. 中心性の可視化


In [ ]:
from src.analysis.visualizer import GraphVisualizer

# 可視化器を初期化
visualizer = GraphVisualizer(output_dir='../results')

# 中心性分布を可視化
visualizer.plot_centrality_distribution(centrality_scores)

# 各中心性タイプでグラフを可視化（上位100ノード）
for centrality_type in ['betweenness', 'closeness', 'eigenvector']:
    if centrality_type in centrality_scores:
        visualizer.plot_graph_with_centrality(
            graph, 
            centrality_scores, 
            centrality_type=centrality_type,
            top_n=100
        )

# 上位ノードの比較
visualizer.plot_top_nodes_comparison(centrality_scores, top_n=20)

print("可視化が完了しました")


## 5. ルーティング可能性分析


In [ ]:
from src.analysis.analyzer import RoutingAnalyzer

# ルーティング分析器を初期化
analyzer = RoutingAnalyzer(graph, centrality_scores)

# 推奨ノードを取得（上位50ノード）
recommended_nodes = analyzer.recommend_nodes_for_channel(top_n=50)

print("推奨ノード（上位20）:")
print(recommended_nodes.head(20).to_string())

# CSVに保存
recommended_nodes.to_csv('../results/recommended_nodes.csv', index=False)
print("\n推奨ノードをCSVに保存しました")


In [ ]:
# 推奨チャネルを取得（上位100チャネル）
# 注意: 全ノードの組み合わせを計算すると時間がかかるため、
# 上位ノードのみを候補として使用
top_nodes = recommended_nodes.head(100)['node_id'].tolist()
recommended_channels = analyzer.recommend_channels(
    candidate_nodes=top_nodes,
    top_n=100
)

print("推奨チャネル（上位20）:")
print(recommended_channels.head(20).to_string())

# CSVに保存
recommended_channels.to_csv('../results/recommended_channels.csv', index=False)
print("\n推奨チャネルをCSVに保存しました")


## 6. 特徴量抽出（機械学習用）


In [ ]:
from src.ml.features import FeatureExtractor

# 特徴量抽出器を初期化
feature_extractor = FeatureExtractor(graph, centrality_scores)

# ノード特徴量を抽出
node_features_df = feature_extractor.create_node_dataframe()
print(f"ノード特徴量: {node_features_df.shape}")
print(node_features_df.head())

# エッジ特徴量を抽出
edge_features_df = feature_extractor.create_edge_dataframe()
print(f"\nエッジ特徴量: {edge_features_df.shape}")
print(edge_features_df.head())

# CSVに保存
node_features_df.to_csv('../results/node_features.csv', index=False)
edge_features_df.to_csv('../results/edge_features.csv', index=False)
print("\n特徴量をCSVに保存しました")


## 7. グラフニューラルネットワーク（GNN）による分析

このセクションでは、AWS SageMakerを使用してGNNモデルを訓練します。
詳細な手順については、`docs/GNN_CENTRALITY_ANALYSIS_GUIDE.md`と`docs/AWS_SAGEMAKER_GUIDE.md`を参照してください。

### 7.1 DGLグラフへの変換


In [ ]:
import dgl
import torch
import numpy as np

# NetworkXグラフをDGLグラフに変換
# 注意: ノードの順序を保持するため、ノードリストを明示的に指定
print("DGLグラフに変換しています...")
node_list = list(graph.nodes())  # ノードの順序を保持
dgl_graph = dgl.from_networkx(graph, node_attrs=None, edge_attrs=None)

# ノード特徴量の準備
num_nodes = dgl_graph.number_of_nodes()

# 基本特徴量（次数）
# 注意: ノードの順序が一致することを確認
degrees = torch.tensor([graph.degree(n) for n in node_list], dtype=torch.float32)
features = degrees.unsqueeze(1)

# 中心性指標を取得（ノードの順序を保持）
betweenness = torch.tensor([centrality_scores['betweenness'].get(n, 0.0) 
                            for n in node_list], dtype=torch.float32)
closeness = torch.tensor([centrality_scores['closeness'].get(n, 0.0) 
                         for n in node_list], dtype=torch.float32)
eigenvector = torch.tensor([centrality_scores['eigenvector'].get(n, 0.0) 
                           for n in node_list], dtype=torch.float32)

# 理論的注意: 中心性指標を特徴量として使用する場合、
# ラベルとして使用しないこと（データリーク防止）
# ここでは、基本特徴量（次数）のみを使用し、中心性は別途保存
# または、中心性を特徴量として使用する場合は、ラベルには使用しない

# オプション1: 基本特徴量のみを使用（推奨: データリークなし）
# features = degrees.unsqueeze(1)

# オプション2: 中心性も特徴量として使用（注意: ラベルには使用しない）
features = torch.cat([features, 
                      betweenness.unsqueeze(1),
                      closeness.unsqueeze(1),
                      eigenvector.unsqueeze(1)], dim=1)

# 特徴量の正規化（重要: 異なるスケールの特徴量を統一）
feature_mean = features.mean(dim=0, keepdim=True)
feature_std = features.std(dim=0, keepdim=True) + 1e-8  # ゼロ除算防止
features_normalized = (features - feature_mean) / feature_std

dgl_graph.ndata['feat'] = features_normalized

# 中心性指標を個別に保存（分析用）
dgl_graph.ndata['betweenness'] = betweenness
dgl_graph.ndata['closeness'] = closeness
dgl_graph.ndata['eigenvector'] = eigenvector

# ラベルの準備（ルーティング可能性）
# 注意: 中心性指標を特徴量として使用している場合、
# ラベルには別の指標（例: 実際のルーティング成功回数など）を使用すべき
# ここでは例として、中心性の組み合わせを使用（実際のデータに置き換えること）
routing_potential = (betweenness * 0.5 + closeness * 0.3 + eigenvector * 0.2)
dgl_graph.ndata['label'] = routing_potential.unsqueeze(1)

print(f"DGLグラフ情報:")
print(f"  ノード数: {dgl_graph.number_of_nodes()}")
print(f"  エッジ数: {dgl_graph.number_of_edges()}")
print(f"  特徴量次元: {features_normalized.shape[1]}")
print(f"  ラベル形状: {dgl_graph.ndata['label'].shape}")
print(f"\n注意: 中心性指標を特徴量として使用している場合、")
print(f"ラベルには実際のルーティングデータを使用することを推奨します。")


### 7.2 グラフデータの保存


In [ ]:
# グラフを保存
dgl.save_graphs('../results/lightning_graph.bin', [dgl_graph])

# メタデータの保存
import json
metadata = {
    'num_nodes': dgl_graph.number_of_nodes(),
    'num_edges': dgl_graph.number_of_edges(),
    'feature_dim': features.shape[1]
}

with open('../results/graph_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("グラフデータを保存しました: ../results/lightning_graph.bin")


## 8. 結果をS3に保存（ローカル実行 + S3保存構成）

**推奨構成**: ローカル（Cursor/Jupyter）で実行し、結果をS3に保存するハイブリッドアプローチ

**利点**:
- **コスト効率**: SageMaker Notebook Instanceの料金がかからない
- **開発速度**: 即座に実行可能、デバッグが容易
- **永続化**: 結果をS3に保存することで、データの永続化と共有が可能
- **柔軟性**: 必要に応じてSageMakerで大規模計算を実行可能

**AWS公式ドキュメント参考**:
- [Use an Amazon S3 bucket for input and output](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-ex-bucket.html)
- [MLCOST04-BP02: Use managed build environments](https://docs.aws.amazon.com/wellarchitected/latest/machine-learning-lens/mlcost04-bp02.html)


In [ ]:
# 分析結果をS3に保存
# ローカル（Cursor/Jupyter）で実行した結果をS3に保存して永続化

import boto3
import os
from pathlib import Path
from datetime import datetime
import yaml

# 設定ファイルからS3バケット情報を取得
try:
    config_path = os.path.join(project_root, 'config', 'config.yaml')
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    bucket_name = config.get('aws', {}).get('s3', {}).get('bucket')
    s3_prefix = config.get('aws', {}).get('s3', {}).get('prefix', 'lightning-network-analysis/')
except Exception as e:
    print(f"設定ファイルの読み込みエラー: {e}")
    bucket_name = None
    s3_prefix = 'lightning-network-analysis/'

# S3クライアントの作成
s3 = boto3.client('s3', region_name='ap-northeast-1')

# タイムスタンプ付きのディレクトリ名を作成
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
s3_results_prefix = f"{s3_prefix}results/{timestamp}/"

# ローカルのresultsディレクトリ
local_results_dir = Path(project_root) / 'results'

if bucket_name:
    print(f"S3バケット: {bucket_name}")
    print(f"S3プレフィックス: {s3_results_prefix}")
    print(f"\n結果をS3にアップロードしています...")
    
    uploaded_files = []
    
    # resultsディレクトリ内のすべてのファイルをアップロード
    if local_results_dir.exists():
        for file_path in local_results_dir.rglob('*'):
            if file_path.is_file():
                # 相対パスを取得
                relative_path = file_path.relative_to(local_results_dir)
                s3_key = f"{s3_results_prefix}{relative_path.as_posix()}"
                
                try:
                    s3.upload_file(str(file_path), bucket_name, s3_key)
                    uploaded_files.append(s3_key)
                    print(f"  ✓ {file_path.name} → s3://{bucket_name}/{s3_key}")
                except Exception as e:
                    print(f"  ✗ {file_path.name} のアップロードに失敗: {e}")
    
    if uploaded_files:
        print(f"\n✓ {len(uploaded_files)}個のファイルをS3にアップロードしました")
        print(f"S3パス: s3://{bucket_name}/{s3_results_prefix}")
    else:
        print("\n⚠️  アップロードするファイルが見つかりませんでした")
        print(f"ローカルのresultsディレクトリ: {local_results_dir}")
else:
    print("⚠️  S3バケット名が設定されていません")
    print("config/config.yamlのaws.s3.bucketを設定してください")
    print("\nローカルに保存されたファイル:")
    if local_results_dir.exists():
        for file_path in local_results_dir.rglob('*'):
            if file_path.is_file():
                print(f"  - {file_path.relative_to(local_results_dir)}")


### 7.3 AWS SageMakerでのGNN訓練（オプション）

**注意**: このセルを実行する前に、以下を確認してください：
1. `config/config.yaml`でAWS設定（IAMロール、S3バケット）を行ってください
2. データをS3にアップロードしてください
3. 詳細は`docs/AWS_SAGEMAKER_GUIDE.md`を参照してください


In [ ]:
# 注意: このセルを実行する前に、config/config.yamlでAWS設定を行ってください
# import boto3
# from src.ml.pipeline import GNNPipeline
# 
# # S3にデータをアップロード
# s3 = boto3.client('s3')
# bucket_name = 'your-bucket-name'  # 実際のバケット名
# 
# s3.upload_file('../results/lightning_graph.bin', bucket_name, 
#                'lightning-network-analysis/data/lightning_graph.bin')
# 
# train_data_path = f's3://{bucket_name}/lightning-network-analysis/data/lightning_graph.bin'
# 
# # GNNパイプラインを初期化
# gnn_pipeline = GNNPipeline()
# 
# # モデルの訓練
# print("GNNモデルの訓練を開始します...")
# estimator = gnn_pipeline.train_gnn_model(
#     train_data_path=train_data_path,
#     hyperparameters={
#         'hidden-dim': 64,
#         'num-layers': 2,
#         'dropout': 0.5,
#         'learning-rate': 0.01,
#         'epochs': 100,
#         'model-type': 'gcn'  # または 'gat'
#     },
#     use_dgl_container=True
# )
# 
# print("GNNモデルの訓練が完了しました")


## 9. 結果のまとめ


In [ ]:
print("=== 分析結果のまとめ ===\n")

print(f"グラフ統計:")
print(f"  ノード数: {graph.number_of_nodes()}")
print(f"  エッジ数: {graph.number_of_edges()}")
print(f"  平均次数: {sum(dict(graph.degree()).values()) / graph.number_of_nodes():.2f}")
print(f"  連結成分数: {nx.number_connected_components(graph)}")

print(f"\n中心性統計:")
for name, scores in centrality_scores.items():
    if scores:
        values = list(scores.values())
        top_node = max(scores.items(), key=lambda x: x[1])
        print(f"  {name}:")
        print(f"    最大値ノード: {top_node[0]}")
        print(f"    最大値: {top_node[1]:.6f}")

print(f"\n推奨ノード数: {len(recommended_nodes)}")
print(f"推奨チャネル数: {len(recommended_channels)}")

print("\n分析が完了しました。結果は results/ ディレクトリに保存されています。")


In [ ]:
# データベース接続を閉じる
db.close()
print("データベース接続を閉じました")
